In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
from IPython.display import clear_output
from operator import itemgetter

import time 

In [ ]:
#Read CSV file containing the agents from SPENSER
df_persons_SPENSER__dir = r'' # use your path
df_persons_SPENSER__file = os.path.join(df_persons_SPENSER__dir, "")
df_persons_SPENSER = pd.read_csv(df_persons_SPENSER__file, index_col=None, header=0)

In [ ]:
# Get only students younger than 16 and older than 2
df_students_SPENSER = df_persons_SPENSER.loc[(df_persons_SPENSER['Age'] <=15)]

In [ ]:
df_students_SPENSER['Age'].max()

In [ ]:
SPENSER_students_ID_list = df_students_SPENSER['PID_AreaMSOA'].unique().tolist()

In [ ]:
len(df_students_SPENSER)

In [ ]:
#Read CSV file containing the matches between NTS and SPENSER:
NTS_SPENSER_matched_dir = r'' # use your path
NTS_SPENSER_matched_file = os.path.join(NTS_SPENSER_matched_dir, "")
df_NTS_SPENSER_matched = pd.read_csv(NTS_SPENSER_matched_file, index_col=None, header=0)

In [ ]:
len(df_NTS_SPENSER_matched)

In [ ]:
## Select only those NTS-SPENSER matches that belong to students
df_NTS_SPENSER_matched_students = df_NTS_SPENSER_matched.loc[(df_NTS_SPENSER_matched['PID_AreaMSOA'].isin(SPENSER_students_ID_list))]

In [ ]:
SPENSER_students_ID_list = df_NTS_SPENSER_matched_students['IndividualID'].unique().tolist()
SPENSER_students_dayID_list = df_NTS_SPENSER_matched_students['DayID'].unique().tolist()

In [ ]:
# Get the OA and MSOA levels from other columns (HID_AreaOA and PID_AreaMSOA)
df_NTS_SPENSER_matched_students['oa'] = df_NTS_SPENSER_matched_students['HID_AreaOA'].str.split('_').str[-1]
df_NTS_SPENSER_matched_students['msoa'] = df_NTS_SPENSER_matched_students['PID_AreaMSOA'].str.split('_').str[-1]

In [ ]:
NTS_SPENSER_matched_students_IndividualID_list = df_NTS_SPENSER_matched_students['IndividualID'].unique().tolist()

In [ ]:
NTS_SPENSER_matched_students_DayID_list = df_NTS_SPENSER_matched_students['DayID'].unique().tolist()

In [ ]:
# Import the file with the days related to each individual
trips_dir = r'' # use your path


df_NTS_trips = pd.read_csv(
    trips_dir,
    sep='\t',
    usecols=['TripID',
             'DayID',               # ID given to all trips made by an individual on a given travel day - Created in SQL
             'IndividualID',        # Individual unique ID - Created in SQL
             'HouseholdID',         # Household unique ID - Created in SQL
             'PSUID',               # PSU unique ID - Created in SQL
             'JourSeq',             # Journey number on a given travel day
             'NumStages',           # Number of stages - actual number
             'MainMode_B04ID',      # Main mode of travel - publication table breakdown - 13 categories
             'TripPurpTo_B01ID',   # Trip purpose - full list - 23 categories
             'TripStartHours',      # Trip start time - hours component
             'TripStartMinutes',    # Trip start time - minutes component
             'TripStart',           # Trip start time - minutes past midnight
             'TripEndHours',        # Trip end time - hours component
             'TripEndMinutes',      # Trip end time - minutes component
             'TripEnd',             # Trip end time - minutes past midnight
             'TripDisIncSW',        # Trip distance - including short walk - miles - actual distance
             'TripTotalTime'        # Total trip time - minutes - actual time
             ]          
)
#persons_in.head()

In [ ]:
## Update transpor mode
mode_mapping = {
    1: 'walk',
     2: 'bike',
     3: 'car',  #'Car/van driver'
     4: 'car_passenger',  #'Car/van passenger'
     5: 'motorcycle',  
     6: 'car',  #'Other private transport',
     7: 'bus', #Bus in London',
     8: 'bus', #'Other local bus',
     9: 'bus', #'Non-local bus',
     10: 'metro', #'London Underground',
     11: 'train', #'Surface Rail',
     12: 'car',  #'Taxi/minicab',
     13: 'metro', #'Other public transport',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['MainMode_B04ID'] = df_NTS_trips['MainMode_B04ID'].map(mode_mapping)

In [ ]:
purp_mapping = {
    1: 'work',
     2: 'work',  #'In course of work',
     3: 'education',
     4: 'food_shop',  #'Food shopping',
     5: 'shop',  #'Non food shopping',
     6: 'medical', #'Personal business medical',
     7: 'eat',  #'Personal business eat/drink',
     8: 'other',  #'Personal business other',
     9: 'eat',  #'Eat/drink with friends',
     10: 'other',  #'Visit friends',
     11: 'leisure',  #'Other social',
     12: 'leisure',  #'Entertain/ public activity',
     13: 'leisure_sport',  #'Sport: participate',
     14: 'home',  #'Holiday: base',
     15: 'leisure_sport',  #'Day trip/just walk',
     16: 'leisure',  #'Other non-escort',
     17: 'escort',  #'Escort home',
     18: 'escort',  #'Escort work',
     19: 'escort',  #'Escort in course of work',
     20: 'escort',  #'Escort education',
     21: 'escort',  #'Escort shopping/personal business',
     22: 'escort',  #'Other escort',
     23: 'home',  #'Home',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['TripPurpTo_B01ID'] = df_NTS_trips['TripPurpTo_B01ID'].map(purp_mapping)

In [ ]:
# Keep only those days that belong to the people selected before (studemts) and purpose = education
df_NTS_trips_students = df_NTS_trips.loc[(df_NTS_trips['IndividualID'].isin(NTS_SPENSER_matched_students_IndividualID_list)) & (df_NTS_trips['DayID'].isin(NTS_SPENSER_matched_students_DayID_list)) & (df_NTS_trips['TripPurpTo_B01ID'] == 'education')]


In [ ]:
len(df_NTS_trips_students)

In [ ]:
#################################

In [ ]:
#############################

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
MSOA_level_centroid_distances_dir = r'' # use your path
MSOA_level_centroid_distances_file = os.path.join(MSOA_level_centroid_distances_dir, "")
df_MSOA_level_centroid_distances = pd.read_csv(MSOA_level_centroid_distances_file, index_col=None, header=0)

In [ ]:
# create a list of the buildings with the following attributes only: 'PID_AreaMSOA', 'Area_MSOA', 'Area_OA' 
df_MSOA_level_centroid_distances = df_MSOA_level_centroid_distances[['msoa_origin', 'msoa_destination', 'distance_miles']]

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
OA_lelvel_centroid_distances_dir = r'' # use your path
OA_lelvel_centroid_distances_file = os.path.join(OA_lelvel_centroid_distances_dir, "")
df_OA_lelvel_centroid_distances = pd.read_csv(OA_lelvel_centroid_distances_file, index_col=None, header=0)

In [ ]:
df_OA_lelvel_centroid_distances.rename(columns={'oa11cd_x': 'oa_origin', 'oa11cd_y': 'oa_destination', 'distance': 'distance_meters'}, inplace=True)

In [ ]:
df_OA_lelvel_centroid_distances['distance_miles'] = df_OA_lelvel_centroid_distances['distance_meters']/1609

In [ ]:
# create a list of the buildings with the following attributes only: 'PID_AreaMSOA', 'Area_MSOA', 'Area_OA' 
df_OA_lelvel_centroid_distances = df_OA_lelvel_centroid_distances[['oa_origin', 'oa_destination', 'distance_miles']]

In [ ]:
#Read CSV file containing the selected NTS days:
education_buildings_dir = r'' # use your path
education_buildings_file = os.path.join(education_buildings_dir, "")
df_education_buildings = pd.read_csv(education_buildings_file, index_col=None, header=0)

In [ ]:
# create a list of the buildings with the following attributes only: 'PID_AreaMSOA', 'Area_MSOA', 'Area_OA' 
education_buildings_list = df_education_buildings[['URN', 'NumberOfPu', 'oa11cd', 'msoa11cd', 'Easting', 'Northing']].values.tolist()

In [ ]:
df_education_buildings['NumberOfPu'].sum()

In [ ]:
# Path to save the results
school_destination_round_1_dataframe = r'' # in case


In [ ]:
# Spilts persons Dataframe into MSOA areas (whole households are stored in OA areas)

grps_persons_oa_split = df_students_SPENSER.groupby('Area_OA')

In [ ]:
t0 = time.time()

oa_num = 0
oa_tot = df_students_SPENSER.Area_OA.nunique()


for oa_name, persons_grp in grps_persons_oa_split:
    
    oa_num += 1
    t1 = time.time()
    clear_output(wait=True)
    print(f'OA num: {oa_num} of {oa_tot}, OA name: {oa_name}')
    print(f'time running {(t1 - t0)/3600}')
    
    for idx_student, student in persons_grp.iterrows():
        

        oa_level = student['Area_OA']
        msoa_level = student['Area_MSOA']
        spenser_id = student['PID_AreaMSOA']

        #print(oa_level)
        #print(msoa_level)
        #print(spenser_id)

        ##########################################################################
        # Get the match between SPENSER student and the NTS person
        df_NTS_SPENSER_matched_student = df_NTS_SPENSER_matched.loc[(df_NTS_SPENSER_matched['PID_AreaMSOA'] == spenser_id)]

        #print(df_NTS_SPENSER_matched_student)

        nts_inidvidual_id = df_NTS_SPENSER_matched_student['IndividualID'].values[0]
        nts_day_id = df_NTS_SPENSER_matched_student['DayID'].values[0]

        #print(nts_inidvidual_id)
        #print(nts_day_id)

        ##########################################################################

        #Find his/her education trips
        df_NTS_trips_student_education = df_NTS_trips_students.loc[(df_NTS_trips_students['IndividualID'] == nts_inidvidual_id) & (df_NTS_trips_students['DayID'] == nts_day_id) & (df_NTS_trips_students['TripPurpTo_B01ID'] == 'education')]


        # If the student has a trip which purpose = 'education':
        # else: do nothing
        if (len(df_NTS_trips_student_education) > 0):


            # If there is more than one, keep only the earliest in the day
            if (len(df_NTS_trips_student_education) > 1):

                df_NTS_trips_student_education = df_NTS_trips_student_education.sort_values('TripStart')

                df_NTS_trips_student_education = df_NTS_trips_student_education.head(1)


            # Get the distance travelled
            trip_distance = df_NTS_trips_student_education['TripDisIncSW'].values[0]



            ##########################################################################
            # Find the OA levels (centroids) that are at the selected or lower distance from the OA level (centroid) where the student lives:
            df_OA_lelvel_centroid_distances_selected = df_OA_lelvel_centroid_distances.loc[(df_OA_lelvel_centroid_distances['oa_origin'] == oa_level) & (df_OA_lelvel_centroid_distances['distance_miles'] <= trip_distance)]

            #print(df_OA_lelvel_centroid_distances_selected)


            OA_list = df_OA_lelvel_centroid_distances_selected['oa_destination'].unique().tolist()


            # Select those education buildings that are located in the previous OA levels and number of remaining puplis is greater than 0
            df_education_buildings_selected = df_education_buildings.loc[(df_education_buildings['oa11cd'].isin(OA_list) & (df_education_buildings['NumberOfPu'] > 0))]




            # Check if the dataframe contains any education building.
            ## If the dataframe is empty, find the education buildings per MSOA level!
            if (len(df_education_buildings_selected) == 0):

                # Find the MSOA levels (centroids) that are at the selected or lower distance from the MSOA level (centroid) where the student lives:
                df_MSOA_lelvel_centroid_distances_selected = df_MSOA_level_centroid_distances.loc[(df_MSOA_level_centroid_distances['msoa_origin'] == msoa_level) & (df_MSOA_level_centroid_distances['distance_miles'] <= trip_distance)]

                #print(df_OA_lelvel_centroid_distances_selected)

                MSOA_list = df_MSOA_lelvel_centroid_distances_selected['msoa_destination'].unique().tolist()

                # Select those education buildings that are located in the previous MSOA levels and number of remaining puplis is greater than 0
                df_education_buildings_selected = df_education_buildings.loc[(df_education_buildings['msoa11cd'].isin(MSOA_list) & (df_education_buildings['NumberOfPu'] > 0))]


            #print(df_education_buildings_selected) 


            ##########################################################################
            # Select randomly one row of the df_education_buildings_selected:
            if (len(df_education_buildings_selected) > 0):
            
                
                ## OLD --> RANDOM SELECTION
                #df_unique_building_education_selected = df_education_buildings_selected.sample(1)
                
                
                ## NEW --> SELECT THE ONE WITH THE HIGHEST CAPACITY
                df_unique_building_education_selected = df_education_buildings_selected[df_education_buildings_selected.NumberOfPu == df_education_buildings_selected.NumberOfPu.max()]
                
                
                #print(df_unique_building_education_selected)

                # Get the relevant information of the selected building:
                building_urn_id = df_unique_building_education_selected['URN'].values[0]
                building_easting = df_unique_building_education_selected['Easting'].values[0]
                building_northing = df_unique_building_education_selected['Northing'].values[0]
                
                
                
                building_oa = df_unique_building_education_selected['oa11cd'].values[0]
                building_msoa = df_unique_building_education_selected['msoa11cd'].values[0]


                # Save information in a list:
                temp_list = []

                temp_list.append(spenser_id)
                temp_list.append(nts_inidvidual_id)
                temp_list.append(nts_day_id)
                temp_list.append(building_urn_id)
                temp_list.append(building_easting)
                temp_list.append(building_northing)
                temp_list.append(building_oa)
                temp_list.append(building_msoa)

                ##########################################################################
                ## Update the number of pupils in the dataframe.

                # Get the number of pupils that can be allocated in the education building
                number_of_pupils = df_unique_building_education_selected['NumberOfPu'].values[0]

                # Remove one unit of the current remainig students to be assigned to that building education
                ## the person selected in the for loop will take this place and should be not accesible for the others.
                df_education_buildings.loc[df_education_buildings['URN'] == building_urn_id, ['NumberOfPu']] = number_of_pupils - 1


                ##########################################################################
                # Save the data in a csv file:

                # Create an empty dataframe
                df_SPENSER_students_education_location = pd.DataFrame([temp_list], columns = ['PID_AreaMSOA', 'IndividualID', 'DayID', 'URN', 'x', 'y', 'oa', 'msoa'])

                # save the data in a csv file
                df_SPENSER_students_education_location.to_csv(school_destination_round_1_dataframe, encoding='utf-8', mode='a', index=False, header=False)


                ##########################################################################


                #if counter == 2:
                    #sys.exit()
        
    
    
    
    
print('Code has been finished. Check results!')  

In [ ]:
t2 = time.time()
print('Time taken (h): ', (t2 - t0) / 3600)

In [ ]:
header_students_matched =["PID_AreaMSOA", "IndividualID", "DayID", "School_PSU", "X", "Y", "oa", "msoa"]

In [ ]:
#Read CSV file containing the agents from SPENSER
school_destination_dataframe_capacity__dir = r'' # use your path
school_destination_dataframe_capacity__file = os.path.join(school_destination_dataframe_capacity__dir, "")
df_school_destination_dataframe_capacity = pd.read_csv(school_destination_dataframe_capacity__file, index_col=None, header=0, names= header_students_matched)

In [ ]:
df_school_destination_dataframe_capacity.head()

In [ ]:
len(df_school_destination_dataframe_capacity)

In [ ]:
len(df_students_SPENSER)

In [ ]:
len(df_students_SPENSER) - len(df_school_destination_dataframe_capacity)

In [ ]:
# % of remaining students to be assigned an education centre
len(df_students_SPENSER_remaining)/len(df_students_SPENSER) * 100

In [ ]:
total_number_pupils_schools = 408275

In [ ]:
# % of students assigned an education centre based on the total capacity of schools
len(df_school_destination_dataframe_capacity)/total_number_pupils_schools * 100

In [ ]:
# % of students assigned an education centre 
len(df_school_destination_dataframe_capacity)/len(df_students_SPENSER) * 100

In [ ]:
spenser_students_ID_matched_list = df_school_destination_dataframe_capacity['PID_AreaMSOA'].unique().tolist()

In [ ]:
df_students_SPENSER_remaining = df_students_SPENSER.loc[~(df_students_SPENSER['PID_AreaMSOA'].isin(spenser_students_ID_matched_list))]

In [ ]:
len(df_students_SPENSER_remaining)

In [ ]:
df_students_SPENSER_remaining.head(30)

In [ ]:
#Read CSV file containing the agents from NTS
df_persons_NTS_dir = r'' # use your path
df_persons_NTS_file = os.path.join(df_persons_NTS_dir, "")
df_persons_NTS = pd.read_csv(df_persons_NTS_file, index_col=None, header=0)

In [ ]:
###############################################################################################################


###############




################################




###############################################################################################################

In [ ]:
# Path to save the results
school_destination_round_1_dataframe = r'' # in case


In [ ]:
school_ID_list = []

for idx_student, student in df_school_destination_dataframe_capacity.iterrows():
    
    school_ID = student['School_PSU']
    
    
    
    if (school_ID not in school_ID_list):
    
        school_ID_list.append(school_ID)
        temp_list = []
        #Find the number of students matched to the school_ID
        
        students_matched_to_school_ID = len(df_school_destination_dataframe_capacity.loc[(df_school_destination_dataframe_capacity['School_PSU'] == school_ID)])
    
    
        #print(students_matched_to_school_ID)
        school_X = student['X']
        school_Y = student['Y']
        
        temp_list.append(school_ID)
        temp_list.append(school_X)
        temp_list.append(school_Y)
        temp_list.append(students_matched_to_school_ID)
        
        
        # Create an empty dataframe
        df_schools_students_matched = pd.DataFrame([temp_list], columns = ['School_PSU', 'X', 'Y', 'Students'])

        # save the data in a csv file
        df_schools_students_matched.to_csv(school_destination_round_1_dataframe, encoding='utf-8', mode='a', index=False, header=False)
        


    
    
print('Code has finished. Check results, amigo')   

In [ ]:
header_school_capacities =["School_PSU", "X", "Y", "pupils"]

In [ ]:
#Read CSV file containing the agents from SPENSER
school_pupils__dir = r'' # use your path
school_pupils__file = os.path.join(school_pupils__dir, "school_destination_round_1_dataframe_capaticty_matched.csv")
df_school_pupils = pd.read_csv(school_pupils__file, index_col=None, header=0, names= header_school_capacities)

In [ ]:
len(df_school_pupils)

In [ ]:
#################################################

In [ ]:
#Read CSV file containing the selected NTS days:
education_buildings_dir = r'' # use your path
education_buildings_file = os.path.join(education_buildings_dir, "education_buildings_studdents_aged_15_or_below_only.csv")
df_education_buildings_2 = pd.read_csv(education_buildings_file, index_col=None, header=0)

In [ ]:
df_education_buildings_2.head()

In [ ]:
# Match schools and compare the % of students matched in each school

In [ ]:
## Merge dataframes to combine df_NTS_SPENSER_matched_msoa with the distances travelled by each individual        
df_school_percentage = pd.merge(df_education_buildings_2, df_school_pupils,  how='left', left_on=['URN'], right_on = ['School_PSU'])


In [ ]:
df_school_percentage.head()

In [ ]:
df_school_percentage = df_school_percentage[['X_x','Y_x', 'URN', 'NumberOfPu','pupils']]

In [ ]:
df_school_percentage.fillna(0)

In [ ]:
df_school_percentage['Percentge_matched'] = df_school_percentage['pupils'] / df_school_percentage['NumberOfPu']*100

In [ ]:
df_school_percentage

In [ ]:
df_school_percentage['Percentge_matched'].min()

In [ ]:
df_school_percentage['Percentge_matched'].max()

In [ ]:
df_school_percentage['Percentge_matched'].mean()

In [ ]:
df_school_percentage['Percentge_matched'].median()

In [ ]:
school_percentage_matched_path = r''

In [ ]:
# save the data in a csv file
df_school_percentage.to_csv(school_percentage_matched_path, encoding='utf-8', mode='a', index=False, header=False)
        

In [ ]:
df_school_percentage['NumberOfPu'].sum()